In [1]:
from __future__ import annotations
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict, Counter
from collections.abc import Callable
from queue import PriorityQueue
from io import StringIO
import seaborn as sns
from dataclasses import dataclass, field
from typing import Any
pd.set_option("display.max_colwidth", 0)

In [2]:
raw_data = pd.read_csv("play_sessions.csv")
raw_data = raw_data[raw_data.user_id.notnull()]
raw_data = raw_data[raw_data.version == "1.0.3"]
raw_data = raw_data.reset_index()
len(raw_data)

619

In [3]:
class Episode():
    def __init__(self):
        self.passing = False
        self.programming_interface = pd.DataFrame()
        self.episode_data = pd.DataFrame()
        self.program = ""
        self.challenge_name = ""
    def __init__(self, pi:pd.DataFrame, ed:pd.DataFrame, passing:bool, program_rep:str, challenge_name:str):
        self.passing = passing
        self.programming_interface = pi
        self.episode_data = ed
        self.program = program_rep
        self.challenge_name = challenge_name
    def __str__(self):
        return str(ed)
        

In [4]:
def parse_raw_data_frames(row:int) -> pd.DataFrame:
    frames = raw_data.frames[row]
    obj = json.loads(frames)
    for i, o in enumerate(obj):
        obj[i] = json.loads(obj[i])
    session = pd.DataFrame(obj)
    return session

memo_frames = {}

def iter_session_frames():
    for i in raw_data.index:
        if i not in memo_frames:   
            frames = parse_raw_data_frames(i)
            memo_frames[i] = frames
        yield memo_frames[i].copy()

def iter_enum_session_frames(): #TODO: uh make this not a weird copy
    for i in raw_data.index:
        yield i, parse_raw_data_frames(i)

In [5]:
organized_sessions = defaultdict(list)
other_actors = pd.DataFrame()

for i, all_frames in iter_enum_session_frames():
    if len(all_frames) == 0:
        continue
    user_id = raw_data.user_id[i]
    
    episode_list = []
    passing = False
    state = ""
    challenge_name = ""
    curr_prog_interface = ""
    prev_prog_interface = ""
    curr_episode_data = ""
    frame_header = all_frames.columns.to_series().to_frame(1).T.to_csv(index=False).partition("\n")[0]

    for i, frame in all_frames.iterrows():
        if frame.actor == "episode_data":
            if frame.object_name == "challenge_pass":
                passing = True
            if frame.verb == "episode_started":
                challenge_name = frame.object_name
                episode_list.append(Episode(
                    passing=passing, 
                    pi=pd.DataFrame(StringIO(f'{frame_header}\n{prev_prog_interface}')), 
                    ed=pd.DataFrame(StringIO(f'{frame_header}\n{curr_episode_data}')), 
                    program_rep=state, challenge_name=challenge_name))
                state = json.dumps(json.loads(frame.state_info["program"]), sort_keys=True)
                passing = False
                prev_prog_interface = curr_prog_interface
                curr_prog_interface = ""
                curr_episode_data = ""
                
            curr_episode_data = f'{curr_episode_data}{frame.to_frame(1).T.to_csv(header=False, index=False)}'
        elif frame.actor == "programming_interface":
            curr_prog_interface = f'{curr_prog_interface}{frame.to_frame(1).T.to_csv(header=False, index=False)}'
        else:
            other_actors = pd.concat([other_actors, frame.to_frame(1).T],  ignore_index=True, sort=True)
    # record last episode and last program changes
    episode_list.append(Episode(
        passing=passing, 
        pi=pd.DataFrame(StringIO(prev_prog_interface)), 
        ed=pd.DataFrame(StringIO(curr_episode_data)), 
        program_rep=state, challenge_name=challenge_name))
    if len(curr_prog_interface) > 0:
        episode_list.append(Episode(
            passing=False, 
            pi=pd.DataFrame(StringIO(curr_prog_interface)), 
            ed=pd.DataFrame(columns=all_frames.columns), 
            program_rep="", challenge_name=challenge_name))
        
    organized_sessions[user_id].append(episode_list)
    

In [6]:
session_metrics = pd.DataFrame(["user_id", "session_index_for_user", "activity_type", "activity_name", 
                        "num_episodes", "passed", "num_episodes_before_passing_or_quitting"])
activity_metrics = pd.DataFrame(["activity_name", "activity_type", "activity_instructions",
                                 "number_of_sessions", "number_of_passing_sessions",
                                "average_episodes_per_session", "median_episodes_per_session",
                                "min_episodes_per_session", "max_episodes_per_session"])

In [7]:
def failing_attempt_counter(list_of_episodes) -> (int, bool):
    counter = 0
    for episode in list_of_episodes:
        if episode.passing:
            return counter, True
        else:
            counter += 1
    return counter, False

session_metric_rows = []

for uid, sessions in organized_sessions.items():
    for i, session in enumerate(sessions):
        num_episodes_before_passing, passing = failing_attempt_counter(session)
        challengetype = "unknown"
        name = "unknown"
        if len(session) > 0:
            name = session[0].challenge_name
            if "try_it" in name:
                challengetype = "try_it"
            elif "direct_instruction" in name:
                challengetype = "direct_instruction"
            elif "mini_challenge" in name:
                challengetype = "mini_challenge"
            elif "challenge" in name:
                challengetype = "challenge"
            else:
                challengetype = "other"
        row = [uid, i, challengetype, name, len(session), passing, num_episodes_before_passing]
        session_metric_rows.append(row)
        
session_metrics = pd.DataFrame(session_metric_rows, 
                               columns=["user_id", "session_index_for_user", "activity_type", "activity_name",
                        "num_episodes", "passed", "num_episodes_before_passing_or_quitting"])
        
        

In [8]:
session_metrics

,user_id,session_index_for_user,activity_type,activity_name,num_episodes,passed,num_episodes_before_passing_or_quitting
0,1923583.0,0,other,spike_curric_turning_in_place_curriculum,2,True,1
1,1923583.0,1,try_it,spike_curric_turning_in_place_left_turn_try_it,2,True,1
2,1923583.0,2,try_it,spike_curric_90_degree_turn_try_it,2,True,1
3,1923583.0,3,other,,1,False,1
4,1923583.0,4,other,,1,False,1
...,...,...,...,...,...,...,...
577,1947126.0,38,try_it,spike_curric_other_turns_sharp_turn_try_it,2,True,1
578,1947126.0,39,mini_challenge,spike_curric_steer_around_the_crater_mini_challenge,21,True,18
579,1947126.0,40,try_it,spike_curric_arm_movement_smaller_movements_try_it,11,True,10
580,1947126.0,41,try_it,spike_curric_arm_movement_getting_stuck_try_it,2,True,1


In [14]:
def find_session(userid:int, challengename:str) -> list:
    for session in reversed(organized_sessions[userid]):
        if len(session) > 0 and session[0].challenge_name == challengename:
            return session
    return []

def print_program_changes_over_time(userid:int=-1, challengename:str=None, episodes:list=None):
    prev_ops = []
    prev_inputs = []
    prev_fields = []
    
    if episodes is None:
        episodes=find_session(userid, challengename)
    
    for episode in episodes:
        if len(episode.program) > 0:
            blocks = json.loads(episode.program)['targets'][0]['blocks']
            ops = [bi['opcode'] for bid, bi in blocks.items()]
            inps = [str(bi['inputs']) for bid, bi in blocks.items()]
            fields = [str(bi['fields']) for bid, bi in blocks.items()]
            print("added:")
            print(Counter(ops) - Counter(prev_ops))
            print(Counter(inps) - Counter(prev_inputs))
            print(Counter(fields) - Counter(prev_fields))
            print("removed:")
            print(Counter(prev_ops) - Counter(ops))
            print(Counter(prev_inputs) - Counter(inps))
            print(Counter(prev_fields) - Counter(fields))
            prev_ops = ops
            prev_inputs = inps
            prev_fields = fields
            print("------------")

In [16]:
print_program_changes_over_time(userid=1927379, challengename="spike_curric_turn_around_craters_mini_challenge")

added:
Counter({'spike_movement_direction_picker': 8, 'spike_movemenet_direction_for_duration': 8, 'spike_movement_moveHeadingForUnits': 6, 'spike_heading_input': 6, 'event_whenprogramstarts': 2})
Counter({'{}': 16, "{'HEADING': [1, 'w{LV=`MU]6(%a2ExP@Fy'], 'RATE': [1, [4, '35']]}": 1, "{'DIRECTION': [1, '%IBmZLo#/*,.u]eQfd!@'], 'RATE': [1, [4, '8']]}": 1, "{'DIRECTION': [1, ',j}lMPXq9/ffO:D9pImu'], 'RATE': [1, [4, '20']]}": 1, "{'DIRECTION': [1, '=s9Sw[qd07jCOz9ZWPZW'], 'RATE': [1, [4, '8']]}": 1, "{'HEADING': [1, 'z%,Jez_H-yw^FRKt}yx1'], 'RATE': [1, [4, '70']]}": 1, "{'DIRECTION': [1, 'RPnWybW}.=lWhplJF~^V'], 'RATE': [1, [4, '35']]}": 1, "{'DIRECTION': [1, 'I$J/@Z#oG3phLss$gGYd'], 'RATE': [1, [4, '8']]}": 1, "{'DIRECTION': [1, 'jzcJ^nFfaf;zXk6pLgBe'], 'RATE': [1, [4, '8']]}": 1, "{'DIRECTION': [1, 's/LO|]yg16!_44Dy6;$J'], 'RATE': [1, [4, '8']]}": 1, "{'HEADING': [1, '9dj5OfSk_Hp]u*lwRN!t'], 'RATE': [1, [4, '30']]}": 1, "{'DIRECTION': [1, 'G;IQ@,t1mp#{ypVsseu`'], 'RATE': [1, [4, '8']]

In [18]:
# all chenges for everyone
for uid, sessions in organized_sessions.items():
    for i, session in enumerate(sessions):
        print("-----------------------------------------------------")
        print_program_changes_over_time(episodes=session)


-----------------------------------------------------
added:
Counter({'spike_movemenet_direction_for_duration': 1, 'event_whenprogramstarts': 1, 'spike_movement_direction_picker': 1})
Counter({'{}': 2, "{'DIRECTION': [1, 'kW^rlmBz~6}C$XdGkH_z'], 'RATE': [1, [4, '10']]}": 1})
Counter({"{'UNITS': ['cm', None]}": 1, '{}': 1, "{'SPIN_DIRECTIONS': ['right', None]}": 1})
removed:
Counter()
Counter()
Counter()
------------
-----------------------------------------------------
added:
Counter({'spike_movemenet_direction_for_duration': 1, 'spike_movement_direction_picker': 1, 'event_whenprogramstarts': 1})
Counter({'{}': 2, "{'DIRECTION': [1, 'dXZkmhT,!)xRKsZzQ}%9'], 'RATE': [1, [4, '10']]}": 1})
Counter({"{'UNITS': ['cm', None]}": 1, "{'SPIN_DIRECTIONS': ['left', None]}": 1, '{}': 1})
removed:
Counter()
Counter()
Counter()
------------
-----------------------------------------------------
added:
Counter({'spike_movement_direction_picker': 1, 'spike_movemenet_direction_for_duration': 1, 'event_w

added:
Counter()
Counter()
Counter()
removed:
Counter()
Counter()
Counter()
------------
added:
Counter()
Counter({"{'SPEED': [1, [4, '100']]}": 1})
Counter()
removed:
Counter()
Counter({"{'SPEED': [1, [4, '200']]}": 1})
Counter()
------------
added:
Counter()
Counter()
Counter()
removed:
Counter()
Counter()
Counter()
------------
added:
Counter()
Counter()
Counter()
removed:
Counter()
Counter()
Counter()
------------
added:
Counter()
Counter()
Counter()
removed:
Counter()
Counter()
Counter()
------------
added:
Counter()
Counter()
Counter()
removed:
Counter()
Counter()
Counter()
------------
added:
Counter()
Counter()
Counter()
removed:
Counter()
Counter()
Counter()
------------
added:
Counter()
Counter()
Counter()
removed:
Counter({'spike_movement_setMovementSpeed': 1})
Counter({"{'SPEED': [1, [4, '100']]}": 1})
Counter({'{}': 1})
------------
added:
Counter()
Counter()
Counter()
removed:
Counter()
Counter()
Counter()
------------
-----------------------------------------------------

In [24]:
# all changes for challenges
for i, row in session_metrics[session_metrics.activity_type == "challenge"].iterrows():
    episodes = organized_sessions[row.user_id][row.session_index_for_user]
    print("-----------------------------------------------------")
    print_program_changes_over_time(episodes=episodes)

-----------------------------------------------------
added:
Counter({'spike_light_turnOnForSeconds': 2, 'note': 1, 'spike_play_beep': 1, 'spike_write': 1, 'event_whenprogramstarts': 1})
Counter({"{'RATE': [1, [5, '2']]}": 2, '{}': 2, "{'BEATS': [1, [4, '0.2']], 'NOTE': [1, 'OK7J`v7|Pe:Mjhkh$9*g']}": 1, "{'TEXT': [1, [10, 'Hello']]}": 1})
Counter({'{}': 3, "{'GRID': [[100, 0, 0, 0, 0, 100, 100, 0, 0, 0, 100, 0, 100, 0, 0, 100, 0, 0, 100, 0, 100, 0, 0, 0, 100], None]}": 1, "{'NOTE': ['60', None]}": 1, "{'GRID': ['100,100,0,100,100,100,100,0,100,100,0,0,0,0,0,100,0,0,0,100,0,100,100,100,0', None]}": 1})
removed:
Counter()
Counter()
Counter()
------------
-----------------------------------------------------
added:
Counter({'note': 9, 'spike_play_beep': 9, 'spike_light_turnOnForSeconds': 2, 'spike_write': 2, 'event_whenprogramstarts': 1})
Counter({'{}': 10, "{'RATE': [1, [5, '2']]}": 2, "{'BEATS': [1, [4, '0.1']], 'NOTE': [1, '|8EkSin$%Rj1xm;u-t?e']}": 1, "{'BEATS': [1, [4, '0.1']], 'NOTE